# Cantera Tutorial: Python

## Getting Started

Open a new Jupyter Notebook. If unfamiliar with Jupyter Notebooks see the [Jupyter Documentation](https://jupyter.readthedocs.io/en/latest/install.html) for installation and basic use instructions. Import the Cantera Python module and NumPy by running:

In [ ]:
import cantera as ct
import numpy as np

When using Cantera, the first thing you usually need is an object representing some phase of matter. Here, we'll create a gas mixture:

In [ ]:
gas1 = ct.Solution('gri30.yaml')

To view the state of the mixture, *call* the `gas1` object as if it were a function:

In [ ]:
gas1()

What you have just done is created an object `gas1` that implements GRI-Mech 3.0, the 53-species, 325-reaction natural gas combustion mechanism developed by Gregory P. Smith, David M. Golden, Michael Frenklach, Nigel W. Moriarty, Boris Eiteneer, Mikhail Goldenberg, C. Thomas Bowman, Ronald K. Hanson, Soonho Song, William C. Gardiner, Jr., Vitali V. Lissianski, and Zhiwei Qin.  See the [GRI-Mech Home Page](http://combustion.berkeley.edu/gri-mech/) for more information.

The `gas1` object has properties you would expect for a gas mixture: a temperature, a pressure, species mole and mass fractions, etc. As we will soon see, it has many more properties.

The summary of the state of `gas1` that you found above shows that the new objects created from the `gri30.xml` input file start out with a temperature of 300 K, a pressure of 1 atm, and have a composition that consists of only one species, in this case hydrogen. There is nothing special about H2 - it just happens to be the first species listed in the input file defining GRI-Mech 3.0. In general, whichever species is listed first will initially have a mole fraction of 1.0, and all others will be zero.

## Setting the State

The state of the object can easily be changed. For example:

In [ ]:
gas1.TP = 1200, 101325

sets the temperature to 1200 K and the pressure to 101325 Pa (Cantera always uses SI Units). After this statement, calling `gas1()` results in:

In [ ]:
gas1()

Notice that the temperature has been changed as requested, but the density has changed too. The pressure and the composition have not.

Thermodynamics generally requires that *two* properties in addition to composition information be specified to fix the intensive state of a substance (or mixture). The state of the mixture can be set using several combinations of two properties. The following are all equivalent:

In [ ]:
gas1.TP = 1200, 101325            # temperature, pressure
gas1.TD = 1200, 0.0204723         # temperature, density
gas1.HP = 1.32956e7, 101325       # specific enthalpy, pressure
gas1.UV = 8.34619e6, 1/0.0204723  # specific internal energy, specific volume
gas1.SP = 85227.6, 101325         # specific entropy, pressure
gas1.SV = 85227.6, 1/0.0204723    # specific entropy, specific volume

In each case, the values of the extensive properties must be entered *per unit mass*.

Properties may be read independently, such as

In [ ]:
gas1.T

or

In [ ]:
gas1.h

or together:

In [ ]:
gas1.UV

The composition can be set in terms of either mole fractions (`X`) or mass fractions (`Y`):

In [ ]:
gas1.X = 'CH4:1, O2:2, N2:7.52'

Mass and mole fractions can also be set using the `dict` object, for cases where the composition is stored in a variable or being computed:

In [ ]:
phi = 0.8
gas1.X = {'CH4':1, 'O2':2/phi, 'N2': 2*3.76/phi}

When the composition alone is changed, the temperature and density are held constant. This means that the pressure and other intensive properties will change. The composition can also be set in conjunction with the intensive properties of the mixture:

In [ ]:
gas1.TPX = 1200, 101325, 'CH4:1, O2:2, N2:7.52'
gas1()

The composition above was specified using a string. The format is a comma-separated list of `<species name>:<relative mole numbers>` pairs. The mole numbers will be normalized to produce the mole fractions, and therefore they are "relative" mole numbers. Mass fractions can be set this way too by changing `X` to `Y` in the above statements.

The composition can also be set using an array, which must have the same size as the number of species. For example, to set all 53 mole fractions to the same value, do this:

In [ ]:
gas1.X = np.ones(53)  # NumPy array of 53 ones

Or, to set all the mass fractions to equal values:

In [ ]:
gas1.Y = np.ones(53)

When setting the state, you can control what properties are held constant by passing the special value `None` to the property setter. For example, to change the specific volume to 2.1 m<sup>3</sup>/kg while holding entropy constant:

In [ ]:
gas1.SV = None, 2.1

Or to set the mass fractions while holding temperature and pressure constant:

In [ ]:
gas1.TPX = None, None, 'CH4:1.0, O2:0.5'

## Getting Help

In addition to the *[Python Module Documentation](https://cantera.org/documentation/index.html)*, documentation of the Python classes and their methods can be accessed from within the Python interpreter as well.

Suppose you have created a Cantera object and want to know what methads are avialable for it, and get help on using the methods:

In [ ]:
g = ct.Solution('gri30.yaml')

To get help on the Python class that this object is an instance of, put a question mark `?` after the variable:

In [ ]:
g?

For a simple list of the properties and methods of this object:

In [ ]:
dir(g)

To get help on a specific method, e.g. the `species_index` method:

In [ ]:
g.species_index?

For properties, getting the documentation is slightly trickier, as the usual method will give you help for the *result*, e.g.:

In [ ]:
g.T?

provides help on Python's `float` class. To get the help for the temperature property, ask for the attribute of the class object itself:

In [ ]:
g.__class__.T?

Help can also be obtained using the `help` function:

In [ ]:
help(g.species_index)

## Chemical Equilibrium

To set a gas mixture to a state of chemical equilibrium, use the `equilibrate` method:

In [ ]:
g = ct.Solution('gri30.yaml')
g.TPX = 300.0, ct.one_atm, 'CH4:0.95, O2:2, N2:7.52'
g.equilibrate('TP')

The above statement sets the state of object `g` to the state of chemical equilibrium holding temperature and pressure fixed. Alternatively, the specific enthalpy and pressure can be held fixed:

In [ ]:
g.TPX = 300.0, ct.one_atm, 'CH4:0.95, O2:2, N2:7.52'
g.equilibrate('HP')
g()